In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
import pickle as pkl
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import pickle as pkl


In [2]:
file1 = open('X_train.pkl','rb')
file2 = open('y_train.pkl','rb')
file3 = open('X_test.pkl','rb')
file4 = open('y_test.pkl','rb')

X_train = pkl.load(file1)
y_train= pkl.load(file2)
X_test = pkl.load(file3)
y_test = pkl.load(file4)

In [5]:
len(y_test)

579

In [4]:
3939 - 3545


394

In [4]:
X_train, X_val, y_train, y_val = train_test_split(
X_train, y_train, test_size=0.1, random_state=42)

train_idx = np.random.permutation(len(X_train))
val_idx = np.random.permutation(len(X_val))

X_train,y_train = X_train[train_idx],y_train[train_idx]
X_val,y_val = X_val[val_idx],y_val[val_idx]

In [5]:
X_train.shape

(3545, 224, 224, 3)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import AveragePooling2D,MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [7]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    fill_mode="nearest"
)

In [8]:
resnet = ResNet50(include_top = False,weights='imagenet',input_shape=(224,224,3))

In [9]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [10]:
for layer in resnet.layers:
    layer.trainable = False
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [11]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32


#model = resnet.output
'''model = Flatten(name="flatten")(model)
model = Dense(128, activation="relu")(model)
model = Dropout(0.5)(model)
model = Dense(1, activation="sigmoid")(model)'''
model = Sequential()
model.add(resnet)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())

model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 100352)            401408    
_________________________________________________________________
batch_normalization_1 (Batch (None, 100352)            401408    
_________________________________________________________________
dense (Dense)                (None, 128)               12845184  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               5

In [14]:
%%time

model.fit(
         aug.flow(X_train,y_train),
         batch_size=BS,
         validation_data=(X_val, y_val),
         epochs=EPOCHS
)

Epoch 1/20
111/111 [==============================] - 178s 2s/step - loss: 0.7797 - accuracy: 0.5625 - val_loss: 0.6752 - val_accuracy: 0.5812
Epoch 2/20
111/111 [==============================] - 178s 2s/step - loss: 0.6945 - accuracy: 0.6110 - val_loss: 0.5491 - val_accuracy: 0.7386
Epoch 3/20
111/111 [==============================] - 177s 2s/step - loss: 0.6523 - accuracy: 0.6403 - val_loss: 0.5254 - val_accuracy: 0.7614
Epoch 4/20
111/111 [==============================] - 176s 2s/step - loss: 0.6428 - accuracy: 0.6494 - val_loss: 0.5528 - val_accuracy: 0.7259
Epoch 5/20
111/111 [==============================] - 176s 2s/step - loss: 0.6193 - accuracy: 0.6629 - val_loss: 0.5315 - val_accuracy: 0.7792
Epoch 6/20
111/111 [==============================] - 176s 2s/step - loss: 0.6132 - accuracy: 0.6736 - val_loss: 0.5759 - val_accuracy: 0.7868
Epoch 7/20
111/111 [==============================] - 176s 2s/step - loss: 0.5988 - accuracy: 0.6753 - val_loss: 0.5431 - val_accuracy: 0.7741

In [15]:
import os.path
if os.path.isfile('models/resnet.h5') is False:
    model.save('models/resnet.h5')

In [19]:
model.evaluate(X_test,y_test)

19/19 [==============================] - 22s 1s/step - loss: 0.8860 - accuracy: 0.8497


[0.8859522938728333, 0.8497409224510193]